In [1]:
import numpy as np
import pandas as pd

In [13]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

In [4]:
df_train = pd.read_csv('Titanic Dataset/train.csv')
df_test = pd.read_csv('Titanic Dataset/test.csv')

In [5]:
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [24]:
X = df_train[['Sex', 'Pclass', 'Embarked']] # categorical variables only
y = df_train['Survived']

In [25]:
clf_steps = [
    ('Imputer', SimpleImputer(strategy='most_frequent')),
    ('Encoding', OneHotEncoder()),
    ('KNN', KNeighborsClassifier())
]

In [26]:
clf_pipeline = Pipeline(clf_steps)
clf_pipeline

Pipeline(steps=[('Imputer', SimpleImputer(strategy='most_frequent')),
                ('Encoding', OneHotEncoder()),
                ('KNN', KNeighborsClassifier())])

In [28]:
cross_val_score(clf_pipeline, X, y, cv=5).mean()

0.7811625133387735

---

In [29]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, OrdinalEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

In [30]:
df_train = pd.read_csv('Titanic Dataset/train.csv')
df_test = pd.read_csv('Titanic Dataset/test.csv')

In [31]:
X = df_train[['Sex', 'Pclass', 'Embarked', 'Age']]
y = df_train['Survived']

In [32]:
num_feature_preprocessing = Pipeline([
    ('MeanImputation', SimpleImputer(strategy='mean')),
    ('Scaler', MinMaxScaler())
])

In [33]:
cat_feature_preprocessing = Pipeline([
    ('ModeImputaion', SimpleImputer(strategy='most_frequent')),
    ('Encoding', OneHotEncoder())
])

In [38]:
pre_processing = ColumnTransformer([
    ('NumFeaturePreprocessing', num_feature_preprocessing, ['Age']),
    ('CatFeaturePreprocessing', cat_feature_preprocessing, ['Sex', 'Pclass', 'Embarked'])],
    remainder='passthrough')

In [39]:
pre_processing

ColumnTransformer(remainder='passthrough',
                  transformers=[('NumFeaturePreprocessing',
                                 Pipeline(steps=[('MeanImputation',
                                                  SimpleImputer()),
                                                 ('Scaler', MinMaxScaler())]),
                                 ['Age']),
                                ('CatFeaturePreprocessing',
                                 Pipeline(steps=[('ModeImputaion',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('Encoding',
                                                  OneHotEncoder())]),
                                 ['Sex', 'Pclass', 'Embarked'])])

In [40]:
clf_pipeline = Pipeline([
    ('Pre_Processing', pre_processing),
    ('KNN', KNeighborsClassifier())
])

In [41]:
clf_pipeline

Pipeline(steps=[('Pre_Processing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('NumFeaturePreprocessing',
                                                  Pipeline(steps=[('MeanImputation',
                                                                   SimpleImputer()),
                                                                  ('Scaler',
                                                                   MinMaxScaler())]),
                                                  ['Age']),
                                                 ('CatFeaturePreprocessing',
                                                  Pipeline(steps=[('ModeImputaion',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('Encoding',
                                                                   OneHotEncoder())]),
                                                  ['Sex', 'Pclass',
                                                   'Embarked'])])),
                ('KNN', KNeighborsClassifier())])

In [42]:
# cross-validate the pipeline
cross_val_score(clf_pipeline, X, y, cv=5).mean()

0.8069863787583955